In [1]:
import pandas as pd
from typing import Literal
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import matplotlib.pyplot as plt
    
from utils.loader import MovieLensDataset
from arquitecture.Recommender import Recommender_2
import torch.optim as optim

SEED = 55
BATCH = 800
NUN_THREADS = 6
DEVICE = "cuda"
NUM_EPOCH = 100
LEARNING_RATE = 0.0005


/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = MovieLensDataset(ml_path="ml-100k", split="train", transpose_ratings=True, seed=SEED)
test_dataset = MovieLensDataset(ml_path="ml-100k", split="test", transpose_ratings=True, seed=SEED)
val_dataset = MovieLensDataset(ml_path="ml-100k", split="val", transpose_ratings=True, seed=SEED)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)

for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
    print("Rating Tensor Shape:", rating_train_tensor.size())
    print("User Data Tensor Shape:", user_data_tensor.size())
    print(f"Max index in rating tensor: {rating_train_tensor.max().item()}")
    print(f"Min index in rating tensor: {rating_train_tensor.min().item()}")
    
    # Get true num_ratings from data
    batch_size, num_ratings_actual, rating_size = rating_train_tensor.size()
    user_batch_size, user_data_dim = user_data_tensor.size()
    
    print(f"Actual num_ratings from tensor: {num_ratings_actual}")
    print(f"Actual user_data_input_dim: {user_data_dim}")
    break

Rating Tensor Shape: torch.Size([678, 22, 19])
User Data Tensor Shape: torch.Size([678, 23])
Max index in rating tensor: 100
Min index in rating tensor: 0
Actual num_ratings from tensor: 22
Actual user_data_input_dim: 23


In [ ]:
def create_model(trial):
    # Hyperparameters to optimize
    ratings_embedding_dim = trial.suggest_int('ratings_embedding_dim', 4, 24)
    ratings_lstm_hidden_size = trial.suggest_int('ratings_lstm_hidden_size', 8, 16)
    ratings_lstm_num_layers = trial.suggest_int('ratings_lstm_num_layers', 1, 16)
    ratings_word_size = trial.suggest_int('ratings_word_size', 8, 16)
    ratings_final_mlp_factor = trial.suggest_int('ratings_final_mlp_factor', 2, 16)
    ratings_embedding_output = trial.suggest_int('ratings_embedding_output', 4, 24)
    
    user_embedding_dim = trial.suggest_int('user_embedding_dim', 8, 16)
    user_embedding_output = trial.suggest_int('user_embedding_output', 8, 16)
    user_factor = trial.suggest_int('user_factor', 2, 16)
    
    final_output_size = 19  # Fixed, since this depends on your target size
    expert_factor = trial.suggest_int('expert_factor', 2, 16)
    
    # Create model with suggested hyperparameters
    max_index_in_data = rating_train_tensor.max().item()
    ratings_num_embeddings = max_index_in_data + 1  # Add 1 since indices are 0-based
    
    model = Recommender_2(
        # For ratings embedder
        ratings_num_embeddings=ratings_num_embeddings,
        ratings_embedding_dim=ratings_embedding_dim,
        ratings_num_ratings=num_ratings_actual,
        ratings_lstm_hidden_size=ratings_lstm_hidden_size,
        ratings_lstm_num_layers=ratings_lstm_num_layers,
        ratings_word_size=ratings_word_size,
        ratings_final_mlp_factor=ratings_final_mlp_factor,
        ratings_embedding_output=ratings_embedding_output,
        # For user embedder
        user_num_embeddings=100,  # Could be optimized too if variable
        user_embedding_dim=user_embedding_dim,
        user_embedding_output=user_embedding_output,
        user_data_input_dim=user_data_dim,
        user_factor=user_factor,
        # Output layer
        final_output_size=final_output_size,
        expert_factor=expert_factor
    ).to(DEVICE)
    
    return model

def objective(trial):
    # Create model with hyperparameters suggested by Optuna
    model = create_model(trial)
    
    # Print model parameters for reference
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Trial {trial.number} - Model parameters: {num_params}")
    
    # Train and evaluate
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss(reduction="sum")
    
    # Track best validation loss
    best_val_loss = float('inf')
    
    for epoch in range(NUM_EPOCH):
        # --- Training Phase ---
        model.train()
        running_train_loss = 0.0
        for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
            optimizer.zero_grad()
            outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE)).to(DEVICE)
            loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE))
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_train_loss = running_train_loss / len(train_dataloader.dataset)
        
        # --- Validation Phase ---
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
                outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE))
                loss = criterion(outputs, rating_test_tensor.to(DEVICE))
                running_val_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
        
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
        
        print(f"Trial {trial.number}, Epoch [{epoch+1}/{NUM_EPOCH}] - "
              f"Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")
        
        # Report intermediate values to Optuna
        trial.report(epoch_val_loss, epoch)
        
        # Handle pruning (early stopping if this trial isn't promising)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return best_val_loss

def evaluate(model: Recommender_2):
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss(reduction="sum")
    
    for epoch in range(NUM_EPOCH):
        # --- Training Phase ---
        model.train()  # Set model to training mode
        running_train_loss = 0.0
        
        for user_data_tensor, rating_train_tensor, rating_test_tensor, _ in train_dataloader:
            optimizer.zero_grad()
            
            outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE)).to(DEVICE)        
            loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE)) 
            
            loss.backward()                 # Backpropagation
            optimizer.step()                # Update model parameters
            running_train_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_train_loss = running_train_loss / len(train_dataloader.dataset)

        # --- Validation Phase ---32
        model.eval()  # Set model to evaluation mode
        running_val_loss = 0.0
        with torch.no_grad():
            for user_data_tensor, rating_train_tensor, rating_test_tensor, _ in val_dataloader:
                
                outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE))        
                loss = criterion(outputs, rating_test_tensor.to(DEVICE)) 
                
                running_val_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCH}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")
    
    return epoch_val_loss

In [ ]:
def run_optimization(n_trials=50):
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1)
    
    print("Best trial:")
    trial = study.best_trial
    
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    # Create the best model
    best_model = create_model(trial)
    
    return best_model, study

In [5]:
# Run the optimization process
best_model, study = run_optimization(n_trials=20)  # Adjust number of trials as needed

# Optionally save the best model
torch.save(best_model.state_dict(), 'best_recommender_model.pt')

# Optionally plot optimization results
try:
    # Plot optimization history
    plot_optimization_history(study)
    plt.savefig('optimization_history.png')
    
    # Plot parameter importances
    plot_param_importances(study)
    plt.savefig('param_importances.png')
except:
    print("Visualization couldn't be generated. Make sure matplotlib is installed.")

# Final evaluation of the best model
final_val_loss = evaluate(best_model)
print(f"Final validation loss with best model: {final_val_loss:.4f}")

[I 2025-04-03 18:47:33,527] A new study created in memory with name: no-name-5c4d5d74-0ec7-4f7f-b772-893c90a10132


Trial 0 - Model parameters: 1522929
Trial 0, Epoch [1/100] - Train Loss: 41.7501, Val Loss: 42.5169
Trial 0, Epoch [2/100] - Train Loss: 41.6412, Val Loss: 42.4149
Trial 0, Epoch [3/100] - Train Loss: 41.5361, Val Loss: 42.3145
Trial 0, Epoch [4/100] - Train Loss: 41.4314, Val Loss: 42.2101
Trial 0, Epoch [5/100] - Train Loss: 41.3215, Val Loss: 42.1001
Trial 0, Epoch [6/100] - Train Loss: 41.2047, Val Loss: 41.9816
Trial 0, Epoch [7/100] - Train Loss: 41.0779, Val Loss: 41.8423
Trial 0, Epoch [8/100] - Train Loss: 40.9272, Val Loss: 41.6712
Trial 0, Epoch [9/100] - Train Loss: 40.7403, Val Loss: 41.4854
Trial 0, Epoch [10/100] - Train Loss: 40.5347, Val Loss: 41.2294
Trial 0, Epoch [11/100] - Train Loss: 40.2488, Val Loss: 40.9106
Trial 0, Epoch [12/100] - Train Loss: 39.8925, Val Loss: 40.4905
Trial 0, Epoch [13/100] - Train Loss: 39.4280, Val Loss: 40.1204
Trial 0, Epoch [14/100] - Train Loss: 39.0014, Val Loss: 39.5605
Trial 0, Epoch [15/100] - Train Loss: 38.3350, Val Loss: 38.933

[I 2025-04-03 18:55:55,224] Trial 0 finished with value: 32.175254821777344 and parameters: {'ratings_embedding_dim': 14, 'ratings_lstm_hidden_size': 11, 'ratings_lstm_num_layers': 15, 'ratings_word_size': 15, 'ratings_final_mlp_factor': 4, 'ratings_embedding_output': 9, 'user_embedding_dim': 14, 'user_embedding_output': 15, 'user_factor': 3, 'expert_factor': 11}. Best is trial 0 with value: 32.175254821777344.


Trial 0, Epoch [100/100] - Train Loss: 30.9951, Val Loss: 32.1956
Trial 1 - Model parameters: 1243613
Trial 1, Epoch [1/100] - Train Loss: 35.2953, Val Loss: 35.2954
Trial 1, Epoch [2/100] - Train Loss: 35.0507, Val Loss: 35.0553
Trial 1, Epoch [3/100] - Train Loss: 34.8191, Val Loss: 34.8154
Trial 1, Epoch [4/100] - Train Loss: 34.5881, Val Loss: 34.5766
Trial 1, Epoch [5/100] - Train Loss: 34.3586, Val Loss: 34.3336
Trial 1, Epoch [6/100] - Train Loss: 34.1258, Val Loss: 34.0855
Trial 1, Epoch [7/100] - Train Loss: 33.8886, Val Loss: 33.8401
Trial 1, Epoch [8/100] - Train Loss: 33.6544, Val Loss: 33.5955
Trial 1, Epoch [9/100] - Train Loss: 33.4213, Val Loss: 33.3545
Trial 1, Epoch [10/100] - Train Loss: 33.1913, Val Loss: 33.0948
Trial 1, Epoch [11/100] - Train Loss: 32.9445, Val Loss: 32.8185
Trial 1, Epoch [12/100] - Train Loss: 32.6836, Val Loss: 32.5401
Trial 1, Epoch [13/100] - Train Loss: 32.4220, Val Loss: 32.2690
Trial 1, Epoch [14/100] - Train Loss: 32.1672, Val Loss: 31.96

[I 2025-04-03 19:04:11,919] Trial 1 finished with value: 22.17156410217285 and parameters: {'ratings_embedding_dim': 9, 'ratings_lstm_hidden_size': 16, 'ratings_lstm_num_layers': 12, 'ratings_word_size': 12, 'ratings_final_mlp_factor': 5, 'ratings_embedding_output': 15, 'user_embedding_dim': 11, 'user_embedding_output': 8, 'user_factor': 3, 'expert_factor': 8}. Best is trial 1 with value: 22.17156410217285.


...

Trial 17, Epoch [19/100] - Train Loss: 35.4760, Val Loss: 35.9073
Trial 18 - Model parameters: 833648


[I 2025-04-03 19:55:18,393] Trial 18 pruned. 


Trial 18, Epoch [1/100] - Train Loss: 45.2949, Val Loss: 45.3610
Trial 19 - Model parameters: 1208709
Trial 19, Epoch [1/100] - Train Loss: 34.0579, Val Loss: 34.6407
Trial 19, Epoch [2/100] - Train Loss: 33.8443, Val Loss: 34.4200
Trial 19, Epoch [3/100] - Train Loss: 33.6318, Val Loss: 34.1979
Trial 19, Epoch [4/100] - Train Loss: 33.4178, Val Loss: 33.9732
Trial 19, Epoch [5/100] - Train Loss: 33.2016, Val Loss: 33.7476
Trial 19, Epoch [6/100] - Train Loss: 32.9847, Val Loss: 33.5008
Trial 19, Epoch [7/100] - Train Loss: 32.7476, Val Loss: 33.2270
Trial 19, Epoch [8/100] - Train Loss: 32.4847, Val Loss: 32.9561
Trial 19, Epoch [9/100] - Train Loss: 32.2248, Val Loss: 32.6917
Trial 19, Epoch [10/100] - Train Loss: 31.9694, Val Loss: 32.3847
Trial 19, Epoch [11/100] - Train Loss: 31.6726, Val Loss: 32.0808
Trial 19, Epoch [12/100] - Train Loss: 31.3797, Val Loss: 31.8339
Trial 19, Epoch [13/100] - Train Loss: 31.1442, Val Loss: 31.5477
Trial 19, Epoch [14/100] - Train Loss: 30.8737, V

[I 2025-04-03 19:57:07,741] Trial 19 pruned. 


Trial 19, Epoch [22/100] - Train Loss: 28.5530, Val Loss: 28.7800
Best trial:
  Value: 16.31423568725586
  Params: 
    ratings_embedding_dim: 22
    ratings_lstm_hidden_size: 10
    ratings_lstm_num_layers: 15
    ratings_word_size: 16
    ratings_final_mlp_factor: 11
    ratings_embedding_output: 19
    user_embedding_dim: 10
    user_embedding_output: 10
    user_factor: 11
    expert_factor: 14
Visualization couldn't be generated. Make sure matplotlib is installed.
Epoch [1/100] - Train Loss: 38.4294, Val Loss: 38.8161
Epoch [2/100] - Train Loss: 38.0001, Val Loss: 38.3945
Epoch [3/100] - Train Loss: 37.5794, Val Loss: 37.9591
Epoch [4/100] - Train Loss: 37.1462, Val Loss: 37.4818
Epoch [5/100] - Train Loss: 36.6712, Val Loss: 37.0306
Epoch [6/100] - Train Loss: 36.2212, Val Loss: 36.5421
Epoch [7/100] - Train Loss: 35.7340, Val Loss: 35.9927
Epoch [8/100] - Train Loss: 35.1864, Val Loss: 35.3384
Epoch [9/100] - Train Loss: 34.5330, Val Loss: 34.5723
Epoch [10/100] - Train Loss: 33